In [1]:
# libraries used
import numpy as np
import pandas as pd
import time

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
# importing the data sets

# reanalysis to train the models, which means that, we will only be using the data from 2015-2017
data = pd.read_csv('reanalysis_merged.csv')

# forecasts to test the models, we will be using only the data from year 2018
data_ensemble = pd.read_csv('EnsembleMean_merged.csv')

In [3]:
# summing the production of energy for each time interval of forecasts
data['shiftedtime'] = pd.NA
for d in data_ensemble.time:
    data.loc[data['time']==d,'shiftedtime'] = d
    
data['marker'] = pd.NA
l = data['shiftedtime'].unique()[1:len(data['shiftedtime'].unique())]
for i,d in enumerate(reversed(l)):
    data.loc[data['time'] < d,'marker'] = i

In [4]:
data.loc[data['marker'].isna(),'marker'] = 11681 # some issues with the last rows
repl = {x:i for i,x in enumerate(data['marker'].unique())}
data['marker2'] = data['marker'].apply(lambda x: repl[x])

data.set_index('time', inplace = True)
cols = [x for x in data.columns if x not in ['shiftedtime', 'marker','time']]
data_reanalysis = data[cols].groupby(['marker2']).mean()

In [5]:
data_reanalysis.index = data.shiftedtime.dropna().index
data_reanalysis['time']=data.shiftedtime.dropna()
data_reanalysis=data_reanalysis.reset_index(drop=True)

In [7]:
data_reanalysis['response']=data_ensemble['response']

In [8]:
data_reanalysis

,norma_10_36.0_-5.0,norma_10_36.0_-5.5,norma_10_36.0_-6.0,norma_10_36.0_-6.5,norma_10_36.5_-5.0,norma_10_36.5_-5.5,norma_10_36.5_-6.0,norma_10_36.5_-6.5,norma_10_37.0_-5.0,norma_10_37.0_-5.5,...,v100_36.5_-5.0,v100_36.5_-5.5,v100_36.5_-6.0,v100_36.5_-6.5,v100_37.0_-5.0,v100_37.0_-5.5,v100_37.0_-6.0,v100_37.0_-6.5,response,time
0,3.488462,2.454596,4.121804,6.880874,3.509729,2.958380,3.976024,6.269876,1.006061,2.484443,...,-4.102994,-4.246074,-7.778730,-7.892693,-3.182035,-3.497699,-6.027443,-5.377989,512.704,2015-01-01 00:00:00
1,2.066014,1.802602,3.822665,7.399457,3.643529,3.039747,4.220490,6.925190,0.838821,3.156342,...,-4.467636,-3.886831,-7.616752,-8.361656,-3.288670,-3.425194,-5.820921,-5.439502,571.841,2015-01-01 03:00:00
2,2.306538,1.507741,2.189116,6.536790,3.760571,2.791709,3.580155,7.410562,1.203006,3.217009,...,-3.415939,-1.038916,-5.775413,-8.287224,-1.119327,-2.419197,-5.652194,-5.622306,443.547,2015-01-01 06:00:00
3,7.005920,5.247110,4.364264,5.817193,4.704203,3.135384,2.646026,6.703552,2.781743,2.721319,...,-1.215549,2.264657,-0.522516,-5.143893,3.419905,-0.870382,-4.810875,-5.449530,391.198,2015-01-01 09:00:00
4,8.986442,8.036524,8.508007,7.867800,6.040125,5.671957,4.508492,5.357550,4.304143,1.671159,...,1.008558,3.234209,2.482362,0.891125,4.949629,1.068143,-1.847648,-2.312948,2201.885,2015-01-01 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11677,9.999894,8.981210,10.815752,11.193703,5.348065,4.737447,4.112121,6.040858,3.455187,2.451645,...,-1.185215,1.904830,0.094201,-2.208365,4.005473,0.090387,-3.289862,-3.474052,1759.303,2018-12-31 09:00:00
11678,8.982202,8.937157,10.997362,11.095194,5.126478,5.291852,4.907039,5.532741,3.908151,2.360333,...,-0.938293,2.094168,1.304973,-0.849058,3.776666,0.378494,-2.230483,-2.924914,2025.514,2018-12-31 12:00:00
11679,8.349431,8.077842,10.373002,10.685829,4.653118,3.853608,3.682994,4.627309,2.752159,1.552905,...,-1.340041,1.544649,1.110487,-0.212592,3.431929,-0.538643,-2.700683,-2.659307,1457.716,2018-12-31 15:00:00
11680,8.203477,7.662909,10.136039,10.830876,4.585443,2.846061,3.116874,4.181634,2.290020,2.630709,...,-1.832549,1.423000,0.749543,-0.285048,4.274322,0.638572,-3.325708,-3.545173,1241.068,2018-12-31 18:00:00


In [9]:
data_reanalysis.to_csv('./reanalysis_merged_3hourMean.csv',index=False)